In [2]:
%pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 17.9 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=746181 sha256=fd53f5b8d86e5e8b7ef4a99d9774c8057dab9e707586ba45bd812a61121b72c9
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm


In [3]:
import time
import os
import zipfile
import csv
import requests
import json
from itertools import islice

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split 

from scipy.sparse import coo_matrix as sp

import warnings # warings 제거
warnings.filterwarnings(action='ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 초기설정시 기본 환경설정

In [4]:
%cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content


In [ ]:
%mkdir steam 

In [ ]:
%cd steam

/content/drive/MyDrive/steam


In [ ]:
!unzip 'steam.zip' 

Archive:  steam.zip
  inflating: parsed_data.csv         
  inflating: parsed_data_only_games.csv  
  inflating: parsed_data_overten.csv  
  inflating: steam_store_data.json   
  inflating: Steam_store_data.pptx   
  inflating: ~$Steam_store_data.pptx  


In [ ]:
# 데이터 로드 (type : dataframe)

In [6]:
with open('/content/drive/MyDrive/steam/steam_store_data.json', 'r') as game_data:
    game_data = pd.read_json(game_data)

In [7]:
with open('/content/drive/MyDrive/steam/parsed_data_overten_reset.csv', 'r') as review_data:
    review_data = pd.read_csv(review_data)

In [8]:
with open('/content/drive/MyDrive/steam/score_data_overten_pos_neg.csv', 'r') as score_data: # ndcg
    score_data = pd.read_csv(score_data)

# 1. Data structure and embedding




> ## Data loading, preprocessing




**user_features** : user의 특성(예시: 성별, 지역, 선호유형)

**item_features**: item의 특성(예시: 제목, 가격, 장르, 국가, 게시 시간)

**interactions** : user-item간의 data(평점)

> ## 데이터프레임 형태 변형 및 확인

In [9]:
game_data = game_data.transpose() # 행렬 뒤집기

In [10]:
game_data = game_data[['steam_appid', 'name']] 
review_data = review_data[['author', 'appid', 'playtime_forever', 'playtime_at_review', 'voted_up', 'votes_up']] 

In [ ]:
review_data.head()

,author,appid,playtime_forever,playtime_at_review,voted_up,votes_up
0,76561198976638577,10,104.0,10,True,0
1,76561198391972225,10,426.0,426,True,1
2,76561198414408167,10,386.0,199,True,0
3,76561198376936422,10,165.0,184,True,0
4,76561198905733625,10,28.0,40,True,0


In [11]:
# 변수명 명시적으로 변경함

ratings = review_data 
games = game_data 

In [12]:
ratings['author'] = ratings['author'].astype('str') # ratings['author']의 타입이 numpy.int64 이므로 str로 변환



> ## Data embedding
위의 데이터를 embedding dataset으로 build



In [ ]:
# 데이터프레임 중복 확인

In [13]:
users = ratings[['author']]
users.drop_duplicates(inplace = True)

In [ ]:
users.head()

,author
0,76561198976638577
1,76561198391972225
2,76561198414408167
3,76561198376936422
4,76561198905733625


In [14]:
ratings.drop_duplicates(inplace = True) # 14916136 -> 13828950로 중복 행 제거

In [15]:
items = ratings[['appid']]
items.drop_duplicates(inplace = True)

In [ ]:
items.head()

,appid
0,10
17438,100
18456,1000010
19601,1000030
20007,1000080


In [ ]:
items.shape[0] # 총 게임 개수는 51927인데 반해 리뷰에 있는 게임 개수는 11566(따라서 interactions의 item 개수는 11566)

11566

In [16]:
games.drop_duplicates(inplace = True) # 51936 -> 51927로 중복 행 제거

In [ ]:
# user-item 사이의 interactions matrix 생성

In [17]:
user_features_source = ratings[['author', 'playtime_forever']]
user_features_source.drop_duplicates(inplace = True)

In [18]:
user_features_source = user_features_source.groupby('author')['playtime_forever'].sum()
user_features_source = pd.DataFrame(user_features_source)
user_features_source.reset_index(inplace = True)

In [19]:
uf = []

playtime_forever = user_features_source[['playtime_forever']]
playtime_forever.drop_duplicates(inplace = True)

for i in range(playtime_forever.shape[0]):
  uf.append('playtime_forever:{0}'.format(playtime_forever.iloc[i][0]))

In [20]:
dataset = Dataset()
dataset.fit(users=users['author'], items=items['appid'],user_features = uf)

In [21]:
ratings_source = ratings[['author', 'appid']]
ratings_source.drop_duplicates(inplace = True)
ratings_source = ratings_source.values.tolist()

In [22]:
interactions, weights = dataset.build_interactions(ratings_source)

In [ ]:
# user_features 빌드

In [23]:
user_features_source = user_features_source[['author', 'playtime_forever']].values.tolist()

In [24]:
user_features_source = [[i[0], ['playtime_forever:{0}'.format(i[1])]] for i in user_features_source]

In [25]:
user_features=dataset.build_user_features(user_features_source)

# 2. LightFM Model Function



> ## Model fit, optimize



loss function 으로  WARP ,  정확도는 precision@k 를 사용하여 evalutation 을 해준다.

In [ ]:
# 모델 생성 및 학습시킴

In [26]:
model = LightFM(loss='warp') 
train, test = random_train_test_split(interactions=interactions, test_percentage=0.25) # test_percentage 0.2 -> 0.25
model.fit(train, item_features=None, user_features=user_features, epochs = 10) # 에포크 2회 -> 10회

In [ ]:
# precision at k 방식으로 평가

In [ ]:
train_precision = precision_at_k(model, train, k=5, item_features=None,user_features=user_features).mean()

KeyboardInterrupt: ignored

In [ ]:
test_precision = precision_at_k(model, test, k=10, item_features=None,user_features=user_features).mean()

In [ ]:
print( 'Precision: train %.4f, test %.4f' 
      % ( train_precision,test_precision ))

In [ ]:
# auc 방식으로 평가

In [ ]:
train_auc = auc_score(model, train, item_features=None,user_features=user_features).mean()

In [ ]:
test_auc = auc_score(model, test, item_features=None,user_features=user_features).mean()

In [ ]:
print( 'AUC: train %.4f, test %.4f.' 
      % ( train_auc, test_auc))

In [98]:
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
from sklearn.metrics import ndcg_score


추천에서는 성능 지표로 precision@k 를 많이 사용하고 

Precision@5 = 0.05 (test) 는 순위로 rank 가 된 게임 중 (k=) 5번째 내에 관심 있을 게임이 나올 정확도가 평균 5% 라는 것입니다. 수치가 매우 낮아보이지만, user/content 수가 많은 real 환경에서 5%를 넘기는 어렵다.
​

AUC는 모델의 전체 적인 성능을 나타내는 지표이다.

​



> ## Predict score & Recommend games 



In [ ]:
labels = items[['appid']]

In [ ]:
def recommendation(model, data, to_recommend): # 인자: model, interactions, to_recommend

    n_users, n_items = data.shape

    scores = model.predict(to_recommend, np.arange(n_items)) 
    labels['scores'] = scores.tolist()
    labels.sort_values(by = ['scores'], ascending = False,inplace = True)
    
    return labels



*   group에 author id를 입력하면 모든 games 의 예측 평점을 계산하고
ex) user 1 -> group = [0], user 2 = group = [1] 
*   list_no에 원하는 게임 수를 입력하면 높은순으로 games(id:score)를 리턴합니다.

In [ ]:
author = str(input('유저 입력: '))

idx = int(users.loc[users['author']== author].index[0]) # predict의 첫번째 인자가 user의 index이므로 변환

유저 입력: 76561198905733625


In [ ]:
list_no = 5 # 임의로 5개 추천

top_items = recommendation(model, interactions, idx)

for x in range(list_no):
  print('name:', games.loc[games['steam_appid'] == top_items.iloc[x][0]].iloc[0][1], '  score:', round(top_items.iloc[x][1], 3)) # 소수점 넷째자리에서 반올림

name: Garry's Mod   score: 4.88
name: Counter-Strike   score: 4.621
name: Counter-Strike: Source   score: 4.589
name: Grand Theft Auto: San Andreas   score: 4.459
name: Counter-Strike: Global Offensive   score: 4.308


In [ ]:
# ndcg 점수 평가

In [31]:
score_data['author'] = score_data['author'].astype('str')

In [34]:
score_data.drop_duplicates(inplace = True)

,Unnamed: 0,author,appid,playtime_forever,playtime_at_review,voted_up,votes_up,time_stamp,scores
0,0,76561198976638577,10,104.0,10,True,0,1625569181,4.155640
1,1,76561198391972225,10,426.0,426,True,1,1625568709,4.178311
2,2,76561198414408167,10,386.0,199,True,0,1625534268,4.175495
3,3,76561198376936422,10,165.0,184,True,0,1625532886,4.159935
4,4,76561198905733625,10,28.0,40,True,0,1625525994,4.150289
...,...,...,...,...,...,...,...,...,...
13829027,13829027,76561198284162593,999660,21.0,21,False,4,1623952510,-1.850732
13829028,13829028,76561198348027001,999660,291.0,200,True,0,1623916123,3.547597
13829029,13829029,76561198250100526,999660,117.0,27,True,0,1623863068,2.454061
13829030,13829030,76561198847043627,999660,606.0,99,True,2,1623855792,5.527272


In [56]:
items.reset_index(inplace = True)

In [201]:
n_users = 100 # interactions.shape[0]에서 세션 다운(램에 제한 존재) 문제 때문에 임의로 100개 유저 기준으로 테스트
k = 5
ndcg_s = []

for i in range(n_users):
  user = users.iloc[i][0]
  appid_for_top_k = score_data.loc[score_data['author'] == user].sort_values('scores', ascending = False)['appid'][:k] # user의 score_data가 존재하는 앱 추출
  n_items = []
  for j in appid_for_top_k:
    n_items.append(items.loc[items['appid'] == j].index[0])
  
  # scores_by_recsys, scores_by_true의 범위가 다르므로 정규화 진행
  scores_by_recsys = np.reshape(model.predict(i, np.asarray(n_items, dtype = np.int32)), (-1, k))
  scores_by_true = np.reshape(np.asarray(score_data.loc[score_data['author'] == user].sort_values('scores', ascending = False)['scores'][:k], dtype = np.float32),  (-1, k))

  norms_by_recsys = np.linalg.norm(scores_by_recsys)
  norms_by_true = np.linalg.norm(scores_by_true)

  scores_by_recsys = scores_by_recsys / norms_by_recsys
  scores_by_true = scores_by_true / norms_by_true
  
  ndcg_s.append(ndcg_score(scores_by_true, scores_by_recsys))

In [202]:
ndcg_s = sum(ndcg_s) / n_users

In [203]:
ndcg_s

0.96064891001017